本书到目前为止一直都在使用命令式编程，**它使用编程语句改变程序状态**。考虑下面这段简单的命令式程序。

In [1]:
def add(a, b):
    return a + b

def fancy_func(a, b, c, d):
    e = add(a, b)
    f = add(c, d)
    g = add(e, f)
    return g

fancy_func(1, 2, 3, 4) # 10

10

和我们预期的一样，在运行语句`e = add(a, b)`时，Python会做加法运算并将结果存储在变量`e`中，从而令程序的状态发生改变。类似地，后面的两条语句`f = add(c, d)`和`g = add(e, f)`会依次做加法运算并存储变量。

虽然使用命令式编程很方便，**但它的运行可能很慢**。一方面，即使`fancy_func`函数中的`add`是被重复调用的函数，Python也会逐一执行这3条函数调用语句。另一方面，**我们需要保存变量`e`和`f`的值直到`fancy_func`中所有语句执行结束**。这是因为在执行`e = add(a, b)`和`f = add(c, d)`这2条语句之后我们并不知道变量`e`和`f`是否会被程序的其他部分使用。

与命令式编程不同，**符号式编程通常在计算流程完全定义好后才被执行**。多个深度学习框架，如**Theano和TensorFlow，都使用了符号式编程**。通常，符号式编程的程序需要下面3个步骤：

1. 定义计算流程；
2. 把计算流程编译成可执行的程序；
3. 给定输入，调用编译好的程序执行。

下面我们用符号式编程重新实现本节开头给出的命令式编程代码。

In [2]:
def add_str():
    return '''
def add(a, b):
    return a + b
'''

def fancy_func_str():
    return '''
def fancy_func(a, b, c, d):
    e = add(a, b)
    f = add(c, d)
    g = add(e, f)
    return g
'''

def evoke_str():
    return add_str() + fancy_func_str() + '''
print(fancy_func(1, 2, 3, 4))
'''

prog = evoke_str()


In [3]:
print(prog)



def add(a, b):
    return a + b

def fancy_func(a, b, c, d):
    e = add(a, b)
    f = add(c, d)
    g = add(e, f)
    return g

print(fancy_func(1, 2, 3, 4))



In [4]:
y = compile(prog, '', 'exec')
exec(y)

10


**以上定义的3个函数都仅以字符串的形式返回计算流程**。最后，我们**通过`compile`函数编译完整的计算流程并运行**。**由于在编译时系统能够完整地获取整个程序，因此有更多空间优化计算**。例如，编译的时候可以将程序改写成`print((1 + 2) + (3 + 4))`，甚至直接改写成`print(10)`。这样不仅**减少了函数调用**，还**节省了内存**。

对比这两种编程方式，我们可以看到以下两点。

* 命令式编程更方便。当我们在Python里使用命令式编程时，大部分代码编写起来都很直观。同时，命令式编程更容易调试。这是因为我们可以很方便地获取并打印所有的中间变量值，或者使用Python的调试工具。

* **符号式编程更高效并更容易移植**。一方面，在编译的时候系统容易做更多优化；另一方面，**符号式编程可以将程序变成一个与Python无关的格式，从而可以使程序在非Python环境下运行，以避开Python解释器的性能问题**。

## 8.1.1 混合式编程取两者之长

大部分深度学习框架在命令式编程和符号式编程之间二选一。例如，Theano和受其启发的后来者`TensorFlow1.x`使用了符号式编程，`Chainer`和它的追随者**`PyTorch`使用了命令式编程**。开发人员在设计`Tensorflow2.x`时思考了这个问题：有没有可能既得到命令式编程的好处，又享受符号式编程的优势？开发者们认为，用户应该用纯命令式编程进行开发和调试；**当需要产品级别的计算性能和部署时，用户可以将大部分命令式程序转换成符号式程序来运行**。Tensorflow通过提供静态图转换器`tf.function`,实现对两种编程方式的支持。在 Tensorflow

在不使用静态图转换器`tf.function`时，用户编写的`python`函数默认会采用命令式编程逐行执行，符合`python`编程的直觉，便于调试，但因为框架不能获得完整的静态运算图，不能进行优化，且动态图不能序列化。官方由此推出了静态图转换器**`tf.function`，其作用在`python_function`后会将这个函数"编译"成一个运算图，接受`input_tensors`为输入并用图执行的方式计算结果，可以加速函数执行，并且可以被序列化后供任何其他语言（如C++和Java）调用**，这样用户只需要在使用动态图运算编写和测试完毕函数之后使用tf.function装饰一下就能够获得静态图的所有优点。

In [21]:
import tensorflow as tf
import numpy as np
from nose.tools import assert_raises

## 8.1.2 tf.function 的使用

### 8.1.2.1 基础

`tf.function`可以定义一个`Tensorflow`操作，既可以命令式的执行它，

In [6]:
@tf.function
def add(a, b):
    return a+b

add(tf.ones([2, 2]), tf.ones([2, 2]))  #  [[2., 2.], [2., 2.]]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 2.],
       [2., 2.]], dtype=float32)>

也可以在图中执行它，并求其梯度，

In [7]:
v = tf.Variable(1.0)
with tf.GradientTape() as tape:
    result = add(v, 1.0)
tape.gradient(result, v)

<tf.Tensor: shape=(), dtype=float32, numpy=1.0>

也可以定义嵌套定义（当然，在实际使用中，可以直接在顶层定义，会自动对子图进行转换），

In [9]:
@tf.function
def dense_layer(x, w, b):
    return add(tf.matmul(x, w), b)

dense_layer(tf.ones([3, 2]), tf.ones([2, 2]), tf.ones([2]))

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[3., 3.],
       [3., 3.],
       [3., 3.]], dtype=float32)>

### 8.1.2.2 追踪与多态

`Python` 的**动态类型意味着用户可以传递多种类型的参数**，这也许会导致函数产生不同的行为。

在另一方面，`Tensorflow`的静态图需要确定的数据类型和维度。**`tf.function` 通过`retracing`函数调用来弥补这一差距**，并在必要的时候产生正确的计算图。`tf.function`大多数微妙的行为都产生自`retracing`行为。

我们可以用不同的参数调用同一函数来观察`retracing`行为：

In [12]:
# Functions are polymorphic

@tf.function
def double(a):
  print("Tracing with", a)   # 跟踪输入变量类型？
  return a + a

print(double(tf.constant(1)))
print()   # 打印空行
print(double(tf.constant(1.1)))
print()
print(double(tf.constant("a")))
print()

Tracing with Tensor("a:0", shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)

Tracing with Tensor("a:0", shape=(), dtype=float32)
tf.Tensor(2.2, shape=(), dtype=float32)

Tracing with Tensor("a:0", shape=(), dtype=string)
tf.Tensor(b'aa', shape=(), dtype=string)



如果希望控制 `tracing` 行为，可以用如下方式操作：

* 创建新的 `tf.function`。**分离 `tf.fucntion` 对象，保证没有共享的计算图引用**。
* 使用 `get_concrete_function` 方法，得到特定的计算图。
* 声明 `input_signature` 当调用 `tf.function` 时，仅跟踪与输入签名一致的调用。

**注：通过例子来理解。**

In [20]:
print("Obtaining concrete trace")
double_strings = double.get_concrete_function(tf.TensorSpec(shape=None, dtype=tf.string))
print()
print("Executing traced function")
print()
print(double_strings(tf.constant("a")))
print()
print(double_strings(a=tf.constant("b")))

print("Using a concrete trace with incompatible types will throw an error")
print()
with assert_raises(tf.errors.InvalidArgumentError):
    double_strings(tf.constant(1))

Obtaining concrete trace

Executing traced function

tf.Tensor(b'aa', shape=(), dtype=string)

tf.Tensor(b'bb', shape=(), dtype=string)
Using a concrete trace with incompatible types will throw an error



In [23]:
@tf.function(input_signature=(tf.TensorSpec(shape=[None], dtype=tf.int32),))
def next_collatz(x):
    print("Tracing with", x)
    return tf.where(x % 2 == 0, x // 2, 3 * x + 1)

print(next_collatz(tf.constant([1, 2])))
# We specified a 1-D tensor in the input signature, so this should fail.
with assert_raises(ValueError):
    next_collatz(tf.constant([[1, 2], [3, 4]]))

Tracing with Tensor("x:0", shape=(None,), dtype=int32)
tf.Tensor([4 1], shape=(2,), dtype=int32)


### 8.1.2.3 追踪触发的时机

多态函数 `tf.function` 会缓存之前追踪行为触发生成过的具体函数。缓存的键由传入的参数确定，对于 `tf.Tensor` 参数而言，是其维度和类型，而对于 `Python` 元语，是其值。对于其它 Python类型，使用对象 id，即对每个不同的类实例都会触发独立的追踪行为，并生成相应的静态图。

### 8.1.2.4 输入参数的选择 Python or Tensor

通常，`Python` 参数被用作超参数，如 `num_layers=10`、`training=True`以及`nonlinearity='relu'`。此时，`Python` 参数的改变触发 `retrace` 行为来构建新的计算图是合理的。然而，在另一些情况下，`Python` 参数并不改变计算图，是**不需要触发 `retrace` 重新构建计算图的**。例如，在训练过程中控制步数，AutoGraph 会自动动态展开，因此传入不同的步数，其生成图是一致的，这时如果触发多个 `trace` 生成同样的计算图，是很低效的。

In [24]:
def train_one_step():
    pass

@tf.function
def train(num_steps):
    print("Tracing with num_steps = {}".format(num_steps))
    for _ in tf.range(num_steps):
        train_one_step()

train(num_steps=10)
train(num_steps=20)

Tracing with num_steps = 10
Tracing with num_steps = 20


一种简单的绕过方式是，**将参数转换为 Tensor**，**这样不改变 shape，就不会触发生成计算图**。

In [25]:
train(num_steps=tf.constant(10))
train(num_steps=tf.constant(20))

Tracing with num_steps = Tensor("num_steps:0", shape=(), dtype=int32)


### 8.1.2.5 tf.function 的附带效应

通常，`Python` 的附带效应（`print` 或改变对象）仅发生在 `tracing` 行为中。何时应该触发 `tf.function` 的附带效应呢？

**经验上推荐仅使用附带效应来 debug `trace` 行为**。其它情况则建议使用 `Tensorflow` 运算如 `tf.Variable.assign`、`tf.print`和`tf.summary`来在 `Tensorflow runtime` 保证代码跟踪和执行。帮助调试的最佳实践是使用函数式风格。

In [26]:
@tf.function
def f(x):
    print("Traced with", x)
    tf.print("Executed with", x)

f(1)
f(1)      # 不生成新的追踪
f(2)

Traced with 1
Executed with 1
Executed with 1
Traced with 2
Executed with 2


如果想要在每次调用 `tf.function` 时执行 `Python` 代码，`tf.py_function` 提供了这种方式的支持。使用 `tf.py_function` 的一个缺点是性能，不能很好的工作在分布式环境下（多GPU/TPU）。由于 `tf.py_function` 需要被可微的连入计算图，输入和输出都会被转换为 `tf.Tensor`。

In [27]:
external_list = []

def side_effect(x):
    print('Python side effect')
    external_list.append(x)

@tf.function
def f(x):
    tf.py_function(side_effect, inp=[x], Tout=[])

f(1)
f(1)
f(1)
assert len(external_list) == 3
# .numpy() call required because py_function casts 1 to tf.constant(1)
assert external_list[0].numpy() == 1

Python side effect
Python side effect
Python side effect


### 8.1.2.6 注意 Python 的状态

许多 Python 特性，如生成器和迭代器，依赖于 Python 运行时跟踪其状态。通常，这些构件在动态图模式下工作正常，但由于 `tracing` 行为，在 `tf.function` 内会发生预期外行为。

例如，迭代器状态被视为一种 Python 附带效应，因此只在 `tracing` 时触发一次。

In [28]:
external_var = tf.Variable(0)
@tf.function
def buggy_consume_next(iterator):
    external_var.assign_add(next(iterator))
    tf.print("Value of external_var:", external_var)

iterator = iter([0, 1, 2, 3])
buggy_consume_next(iterator)
# This reuses the first value from the iterator, rather than consuming the next value.
buggy_consume_next(iterator)
buggy_consume_next(iterator)

Value of external_var: 0
Value of external_var: 0
Value of external_var: 0


如果一个迭代器生成和消费完全在 `tf.function` 中，它会正确地工作，但会产生巨大地计算图，这也许不符合你的预期，更严重的是，对于以 Python List 表示的内存中大型数据集，相应的大型计算图也并不能带来性能提升。

如果想要在 `Python` 数据集上迭代，最安全的方式是使用 `tf.data.Dataset` 封装，并以 `for x in y` 方式遍历。AutoGraph 对 `for` 循环中 `y` 是一个 `tf.Tensor` 或 `tf.data.Dataset` 有特殊支持。

In [31]:
def measure_graph_size(f, *args):
    g = f.get_concrete_function(*args).graph
    print("{}({}) contains {} nodes in its graph".format(
        f.__name__, ', '.join(map(str, args)), len(g.as_graph_def().node)))

@tf.function
def train(dataset):
    loss = tf.constant(0)
    for x, y in dataset:
        loss += tf.abs(y - x) # Some dummy computation.
    return loss

small_data = [(1, 1)] * 2
big_data = [(1, 1)] * 10
measure_graph_size(train, small_data)
measure_graph_size(train, big_data)

train([(1, 1), (1, 1)]) contains 8 nodes in its graph
train([(1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1)]) contains 32 nodes in its graph


In [30]:
measure_graph_size(train, tf.data.Dataset.from_generator(
    lambda: small_data, (tf.int32, tf.int32)))
measure_graph_size(train, tf.data.Dataset.from_generator(
    lambda: big_data, (tf.int32, tf.int32)))

train(<FlatMapDataset shapes: (<unknown>, <unknown>), types: (tf.int32, tf.int32)>) contains 8 nodes in its graph
train(<FlatMapDataset shapes: (<unknown>, <unknown>), types: (tf.int32, tf.int32)>) contains 8 nodes in its graph


当封装 `Python/Numpy` 数据为 `tf.data.Dataset` 时，要注意 `tf.data.Dataset.from_generator` 与 `tf.data.Dataset.from_tensors` 的区别。**前者**保持数据在 `Python runtime` 中，通过 `tf.py_function` 取数，**会造成一定的性能瓶颈**。而**后**者会复制一份到 `Tensorflow runtime` 成为计算图的一个 `tf.constant()` 节点，**会占用更多的内存**。

从文件中读数据，如 `TFRecordDataset/CsvDataset` 等，**是读取数据最高效的方式**，`Tensorflow` 可以自行管理异步读取和预加载数据，而不需要 `Python` 的参与。

### 8.1.2.7 自动控制依赖

作为编程模型，`tf.function` 一个非常吸引人的特性是，**在通常的数据流图之上，为运行时环境提供了更多关于代码预期行为的信息**。

例如，当写代码是多次读写同一变量，数据流图也许不会编码原本预期的操作顺序。在 `tf.function` 中，则会依照在 `Python` 代码中的声明顺序消除执行顺序的歧义。这使得 `tf.function` 支持状态操作，可以复制动态图模式的语义。

这意味着不再需要手动添加控制依赖，而**可以交由 `tf.function` 自动添加控制依赖**。

In [32]:
# Automatic control dependencies

a = tf.Variable(1.0)
b = tf.Variable(2.0)

@tf.function
def f(x, y):
    a.assign(y * b)
    b.assign_add(x * a)
    return a + b

f(1.0, 2.0)  # 10.0

<tf.Tensor: shape=(), dtype=float32, numpy=10.0>

In [33]:
@tf.function
def f(x):
    v = tf.Variable(1.0)
    v.assign_add(x)
    return v

with assert_raises(ValueError):
    f(1.0)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [34]:
v = tf.Variable(1.0)

@tf.function
def f(x):
    return v.assign_add(x)

print(f(1.0))  # 2.0
print(f(2.0))  # 4.0

tf.Tensor(2.0, shape=(), dtype=float32)
tf.Tensor(4.0, shape=(), dtype=float32)


只要可以证明`tf.function`中的变量只在函数初次执行时被创建，也是可以通过检查的。

In [35]:
class C:
    pass

obj = C()
obj.v = None

@tf.function
def g(x):
    if obj.v is None:
        obj.v = tf.Variable(1.0)
    return obj.v.assign_add(x)

print(g(1.0))  # 2.0
print(g(2.0))  # 4.0

tf.Tensor(2.0, shape=(), dtype=float32)
tf.Tensor(4.0, shape=(), dtype=float32)


In [38]:
tf.random.uniform([5])

<tf.Tensor: shape=(5,), dtype=float32, numpy=
array([0.78828204, 0.5091201 , 0.19778812, 0.8517325 , 0.83865917],
      dtype=float32)>

In [39]:
# Simple loop

@tf.function
def f(x):
    while tf.reduce_sum(x) > 1:
        tf.print(x)
        x = tf.tanh(x)
    return x

f(tf.random.uniform([5]))

[0.391111255 0.812969923 0.0989073515 0.647075057 0.322456956]
[0.372317851 0.671225369 0.0985860825 0.569697618 0.311726809]
[0.356017441 0.585785389 0.0982679278 0.515137196 0.302007198]
[0.341701269 0.526857615 0.0979528353 0.473938167 0.293148398]
[0.328995347 0.482975364 0.0976407602 0.441375792 0.28503]
[0.317617744 0.448623329 0.0973316506 0.414784193 0.277554035]
[0.307351261 0.420766741 0.0970254615 0.392527342 0.270639777]
[0.298025519 0.397576183 0.0967221409 0.373536974 0.264220029]
[0.289504647 0.377873152 0.0964216515 0.357081592 0.258238405]
[0.281678826 0.360859 0.0961239487 0.342640817 0.252647102]
[0.27445814 0.345970452 0.0958289877 0.329832941 0.24740535]
[0.267768234 0.33279708 0.0955367237 0.31837064 0.242478102]
[0.26154694 0.321031809 0.0952471197 0.3080329 0.23783505]
[0.255741864 0.310439587 0.0949601382 0.29864648 0.233449817]
[0.250308454 0.300836951 0.0946757346 0.290073454 0.229299352]
[0.245208591 0.292078346 0.0943938717 0.282202423 0.225363389]
[0.24040

<tf.Tensor: shape=(5,), dtype=float32, numpy=
array([0.21655877, 0.24676272, 0.09248893, 0.24073401, 0.20253736],
      dtype=float32)>

如下代码可以观察 `autograph` 生成的代码

In [41]:
def f(x):
    while tf.reduce_sum(x) > 1:
        tf.print(x)
        x = tf.tanh(x)
    return x

print(tf.autograph.to_code(f))

def tf__f(x):
    do_return = False
    retval_ = ag__.UndefinedReturnValue()
    with ag__.FunctionScope('f', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:

        def get_state():
            return (x,)

        def set_state(loop_vars):
            nonlocal x
            (x,) = loop_vars

        def loop_body():
            nonlocal x
            ag__.converted_call(tf.print, (x,), None, fscope)
            x = ag__.converted_call(tf.tanh, (x,), None, fscope)

        def loop_test():
            return (ag__.converted_call(tf.reduce_sum, (x,), None, fscope) > 1)
        ag__.while_stmt(loop_test, loop_body, get_state, set_state, ('x',), {})
        try:
            do_return = True
            retval_ = fscope.mark_return_value(x)
        except:
            do_return = False
            raise
    (do_return,)
    return ag__.retval(retval_)



### 8.1.3.1 AutoGraph：条件分支

`AutoGraph` 将 `if` 语句转换为等效的 `tf.cond` 调用。**这一替换发生在条件变量为张量时**，除此之外的条件，在 `tracing` 时确定。

`test_tf_cond` 函数用来检查函数中是否使用了 `tf.cond`。

In [42]:
def test_tf_cond(f, *args):
    g = f.get_concrete_function(*args).graph
    if any(node.name == 'cond' for node in g.as_graph_def().node):
          print("{}({}) uses tf.cond.".format(
              f.__name__, ', '.join(map(str, args))))
    else:
          print("{}({}) executes normally.".format(
              f.__name__, ', '.join(map(str, args))))

    print("  result: ",f(*args).numpy())

当参数为 python `True` 时，正常地执行条件：

In [43]:
@tf.function
def dropout(x, training=True):
    if training:
        x = tf.nn.dropout(x, rate=0.5)
    return x

test_tf_cond(dropout, tf.ones([10], dtype=tf.float32), True)

dropout(tf.Tensor([1. 1. 1. 1. 1. 1. 1. 1. 1. 1.], shape=(10,), dtype=float32), True) executes normally.
  result:  [0. 0. 0. 0. 0. 0. 0. 0. 2. 2.]


但传递一个张量则会使 python `if` 替换为 `tf.cond`：

In [44]:
test_tf_cond(dropout, tf.ones([10], dtype=tf.float32), tf.constant(True))

dropout(tf.Tensor([1. 1. 1. 1. 1. 1. 1. 1. 1. 1.], shape=(10,), dtype=float32), tf.Tensor(True, shape=(), dtype=bool)) uses tf.cond.
  result:  [2. 2. 2. 0. 2. 0. 2. 0. 2. 2.]


### 8.1.3.2 AutoGraph 与循环

`AutoGraph` 有一些转换循环的简单规则。

* `for`：迭代器是张量时转换
* `while`：循环条件与张量有关时转换

如果一个循环被转换，它将由 `tf.while` 动态展开，或在 `for x in tf.data.Dataset` 情况下，将循环转换为 `tf.data.Dataset.reduce`。

如果循环没有被转换，则静态展开。

`test_dynamically_unrolled(f, *args)`

In [45]:
def test_dynamically_unrolled(f, *args):
    g = f.get_concrete_function(*args).graph
    if any(node.name == 'while' for node in g.as_graph_def().node):
        print("{}({}) uses tf.while_loop.".format(
          f.__name__, ', '.join(map(str, args))))
    elif any(node.name == 'ReduceDataset' for node in g.as_graph_def().node):
        print("{}({}) uses tf.data.Dataset.reduce.".format(
          f.__name__, ', '.join(map(str, args))))
    else:
        print("{}({}) gets unrolled.".format(
          f.__name__, ', '.join(map(str, args))))

#### For 循环

`tf.function` 的静态展开

In [46]:
@tf.function
def for_in_range():
    x = 0
    for i in range(5):
        x += i
    return x

test_dynamically_unrolled(for_in_range)

for_in_range() gets unrolled.


In [49]:
@tf.function
def for_in_tfrange():
    x = tf.constant(0, dtype=tf.int32)
    for i in tf.range(5):
        x += i
    return x

test_dynamically_unrolled(for_in_tfrange)

for_in_tfrange() uses tf.while_loop.


In [50]:
@tf.function
def for_in_tfdataset():
    x = tf.constant(0, dtype=tf.int64)
    for i in tf.data.Dataset.range(5):
        x += i
    return x

test_dynamically_unrolled(for_in_tfdataset)

for_in_tfdataset() uses tf.data.Dataset.reduce.


While 循环

In [51]:
@tf.function
def while_py_cond():
    x = 5
    while x > 0:
        x -= 1
    return x

test_dynamically_unrolled(while_py_cond)

while_py_cond() gets unrolled.


In [52]:
@tf.function
def while_tf_cond():
    x = tf.constant(5)
    while x > 0:
        x -= 1
    return x

test_dynamically_unrolled(while_tf_cond)

while_tf_cond() uses tf.while_loop.


> 感兴趣的可以去看[原文](https://zh.d2l.ai/chapter_computational-performance/hybridize.html)

> [Better performance with tf.function](https://www.tensorflow.org/tutorials/customization/performance)

> [Functions, not Sessions](https://github.com/tensorflow/community/blob/master/rfcs/20180918-functions-not-sessions-20.md)

> [AutoGraph Reference](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/autograph/g3doc/reference/limitations.md)

> [tf.Module](https://github.com/tensorflow/community/blob/master/rfcs/20190117-tf-module.md)

> [Tensorflow2.0 学习笔记之静态图转换器](https://zhuanlan.zhihu.com/p/74441082)

> [Tensorflow2.0 学习笔记之状态容器](https://zhuanlan.zhihu.com/p/73575776)

